In [ ]:
import os
import pygame
import pyautogui
import pywhatkit
from datetime import datetime
import time
import speech_recognition as srp
import Youtube_skip
import requests
import pyfiglet


#Rose name display in ASCII format
styled_text=pyfiglet.figlet_format('R O S E..!',font= '3-d')
print(styled_text)
# print(pyfiglet.figlet_format("R O S E..."))
def speak(text):
# Rose voice
    voice = "en-US-AriaNeural"

    # Build the command for the edge-tts tool
    command = f'edge-tts --voice "{voice}" --text "{text}" --write-media "audio/output.mp3"'

    # Run the command using os.system
    os.system(command)

    # Initialize pygame and pygame.mixer
    pygame.init()
    pygame.mixer.init()

    try:
        # Load the audio file into the mixer
        pygame.mixer.music.load("audio/output.mp3")

        # Play the loaded audio file
        pygame.mixer.music.play()

        # Wait until the audio is finished playing
        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(5)

    except Exception as e:
        # Print any exceptions that occur
        print(e)

def command_from_user():
    rec = srp.Recognizer()

    with srp.Microphone() as source:
        speak("I'm listening. please speak!")
        rec.pause_threshold = 0.5 #if pause if more than .5sec it assume speech is finish and start searching 

        try:
            audio = rec.listen(source)
            print("Analyzing the voice...!")
            query_general = rec.recognize_google(audio, language='en-us')

        except srp.UnknownValueError:
            # Handle the case where speech recognition could not understand the audio
            speak("Sorry, I couldn't understand what you said. Please try again.")
            return ""

        except srp.RequestError as e:
            # Handle the case where there is an error with the Google Speech Recognition service
            print(f"Error with the Google Speech Recognition service: {e}")
            return ""

        except Exception as e:
            # Handle any other unexpected errors
            print(f"An unexpected error occurred: {e}")
            return ""

    # Return the recognized query_general
    return query_general

def greet_user():
    current_time = datetime.now().hour
    if 3 <= current_time < 12:
        speak("Good morning, sir!")
    elif 12 <= current_time < 18:
        speak("Good afternoon, sir!")
    else:
        speak("Good evening, sir!")

def rose_intro():
    speak("I'm Your Personal AI, My name is Rose")
    speak("Hye Shivam, How can I assist you today....?")

def process_open_command(query_general):
    app_name = query_general.replace('open', '')
    speak(f'opening {app_name}')
    with pyautogui.hold('command'):
        pyautogui.press('space')
    pyautogui.typewrite(app_name)
    pyautogui.sleep(0.2)
    pyautogui.press('enter')

def process_switch_tab_command_right(query_general):
    pyautogui.hotkey('Ctrl','tab')
    time.sleep(2)
    speak('tab is switched sir')
    
def process_switch_tab_command_left(query_general):
    pyautogui.hotkey('Ctrl','shift','tab')
    time.sleep(2)
    speak('tab is switched sir')

def process_close_tab_command(query_general):
    with pyautogui.hold('command'):
        pyautogui.press('w')
    time.sleep(0.5)
    speak('tab is closed sir')

def process_close_app_command(query_general, app_name):
    with pyautogui.hold('command'):
        time.sleep(4)
        pyautogui.press('q')
    time.sleep(4)
    speak(f'{app_name} is closed, sir')

def process_play_command(query_general):
    song_name = query_general.replace('play', '')
    try:
        speak(f'As your command, sir. Playing {song_name} for you.')
        pywhatkit.playonyt(song_name)
        time.sleep(10)
        #click_skip_ad_button()

    except Exception as e:
        speak("Sorry, there was an issue playing the requested song")
    
    # speak(f'As your command, sir. Playing {song_name} for you.')
    # pywhatkit.playonyt(song_name)
    time.sleep(10)  # Assuming 10 seconds for the video to start, adjust as needed
    # click_skip_ad_button()
    
def get_current_time():
    current_time = datetime.now().strftime('%I:%M %p')
    return current_time

def process_time_command():
    current_time = get_current_time()
    speak(f'The current time is {current_time}')

# Additional functions for handling other commands...

def click_skip_ad_button():
    try:
        ad_text_path = '//*[@id="ad-text:7"]' # XPath for skip ad
        skip_button_location = pyautogui.locateCenterOnScreen(ad_text_path)
        if skip_button_location is not None:
            pyautogui.click(skip_button_location)
        else:
            speak("Skip button not found on the screen.")
    except Exception as e:
        print("Error clicking the skip ad button:", e)

def news():
    news_api = 'https://newsapi.org/v2/top-headlines?country=us&apiKey=b1cd548d0ee540f89978bee09b0bba8a'
    
    main_page = requests.get(news_api).json()
    article = main_page["articles"]
    heading=[]
    day=["first","second","third","fourth",'fifth',"sixth","seventh","eighth","ninth","tenth"]

    for ar in article:
        heading.append(ar["title"])
    for i in range(len(day)):
        speak(f"today's {day[i]} news is: {heading[i]}")
        
###############################################################################################################        

# query_youtube_skip = ""
def main():
    app_name = ""

    greet_user()
    rose_intro()

    while True:
        query_general = command_from_user().lower()

        print('\nYou said (General):', query_general)
        #print('\nYou said (YouTube skip):', query_youtube_skip)

        if 'open' in query_general:
            process_open_command(query_general)

        elif 'switch right' in query_general:
            process_switch_tab_command_right(query_general)

        elif 'switch left' in query_general:
            process_switch_tab_command_left(query_general)
            
        elif 'close tab' in query_general:
            process_close_tab_command(query_general)

        elif f'close the {app_name}' in query_general:
            process_close_app_command(query_general, app_name)

        elif 'play' in query_general:
            process_play_command(query_general)
        
        elif 'time' in query_general:
            process_time_command()
            
        elif 'tell me the today news' in query_general:
            speak("Okay sir!, gathering the latest news")
            news()

        # Additional conditions for other commands...

        elif any(keyword in query_general for keyword in ['stop', 'exit', 'quit', 'sunshine', 'by rose']):
            speak('Thank you for using rose ai, have a great day, sir!')
            speak('If you need assistance in the future, feel free to call upon me. Goodbye!')
            break

        else:
            # speak("I'm not sure how to handle that, sir")
            pass

    # Cleanup code if needed
    pygame.mixer.music.stop()
    pygame.mixer.quit()

if __name__ == "__main__":
    main()


In [ ]:
def take_command():
    r = srp.Recognizer()
    with srp.Microphone() as source:
        print('listening')
        r.pause_threshold=1
        audio = r.listen(source)

    try:
        print('Recognizing')
        query = r.recognize_google(audio, language='en-US')

    except Exception as e:
        print(e)
        return ""

    return query